In [1]:
import os
import pandas as pd
import boto3

In [2]:
def get_image(imageFile,client):
    try:
        with open(imageFile, 'rb') as image:
                return client.detect_labels(Image={'Bytes': image.read()},MaxLabels=5,MinConfidence=60)
    except:
        print(f'{imageFile} n\' est pas une image')
        d = ''
        return d    

In [3]:
def get_aws_labels(datadir,datadirtemp,df,start,client):
    print(f'proceeding at start point {start} | df.index length {len(df.index)}')
    
    df_aws = pd.DataFrame()
    
    for index, row in df.iterrows():
        file = datadir+row.file_name
        #print(f'file {file} index {index}')
        response = get_image(file,client)
        
        if type(response) is dict:
            label_res = pd.DataFrame(response['Labels'])
            label_res['cle'] = index

            if len(label_res.index > 0):
                label_res = label_res.pivot(index='cle', columns='Name', values='Confidence')
                df_aws = pd.concat([df_aws, label_res], axis=0, ignore_index=False, sort=True)
    
    df = pd.merge(df,df_aws,left_index=True,right_index=True, how='outer')
    
    csv_file =datadirtemp+'temp'+str(start).zfill(5)+'.csv' 
    df.to_csv(csv_file,sep=';')
    print(f'start point {start} | df.index length {len(df.index)} | csv {csv_file} writen')
    
    return start+len(df.index)

In [4]:
if __name__ == "__main__":
    
    #recupération de la liste des fichiers à traiter avec AWS REKO
    datadir = "./Upload/"
    datadirtemp = "./collect_temp2/"
    client=boto3.client('rekognition')

    #parcourir dataset reference par pack de 100
    df_ref = pd.read_csv('Mairie_appels_filename.csv',sep=';')
    batch_size = 100
    start = 0
    
    if True:
        length = len(df_ref.index)
    else:
        length = 10
    
    while start < length:
        start = get_aws_labels(datadir,datadirtemp,df_ref.loc[start:start+batch_size],start,client)
    

proceeding at start point 0 | df.index length 101
start point 0 | df.index length 101 | csv ./collect_temp2/temp00000.csv writen
proceeding at start point 101 | df.index length 101
start point 101 | df.index length 101 | csv ./collect_temp2/temp00101.csv writen
proceeding at start point 202 | df.index length 101
start point 202 | df.index length 101 | csv ./collect_temp2/temp00202.csv writen
proceeding at start point 303 | df.index length 101
start point 303 | df.index length 101 | csv ./collect_temp2/temp00303.csv writen
proceeding at start point 404 | df.index length 101
start point 404 | df.index length 101 | csv ./collect_temp2/temp00404.csv writen
proceeding at start point 505 | df.index length 101
start point 505 | df.index length 101 | csv ./collect_temp2/temp00505.csv writen
proceeding at start point 606 | df.index length 101
start point 606 | df.index length 101 | csv ./collect_temp2/temp00606.csv writen
proceeding at start point 707 | df.index length 101
start point 707 | df.